## Deploying a Decision Optimization model with Watson Machine Learning

This notebook shows you how to deploy a Decision Optimization model, create and monitor jobs, and get solutions using the Watson Machine Learning Python Client.


## Table of Contents
1. [Install the Watson Machine Learning client API](#setup)
2. [Create a client instance](#create)
3. [Prepare your model archive](#prepare)
4. [Upload your model on Watson Machine Learning](#upload)
5. [Create a deployment](#deploy)
6. [Create and monitor a job with inline data for your deployed model](#job)
7. [Display the solution](#display)
8. [Solve another problem using the same deployment](#problem)
9. [Summary](#summary)

<a id='setup'></a>
### Set up the Watson Machine Learning client

Before you use the sample code in this notebook, you need to:

- create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener noreferrer">Watson Machine Learning (WML) Service</a> instance. A free plan is offered and information about how to create the instance can be found at <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-setup.html" target="_blank" rel="noopener noreferrer"> https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html.</a>


Install and then import the Watson Machine Learning client library. This notebook uses the preview Python client based on v4 of Watson Machine Learning APIs. 

**Important** Do not load both Python client libraries into a notebook.

In [1]:
# Uninstall the Watson Machine Learning client Python client based on v3 APIs

!pip uninstall watson-machine-learning-client -y

Uninstalling watson-machine-learning-client-1.0.371:
  Successfully uninstalled watson-machine-learning-client-1.0.371


In [2]:
# Install the WML client API

!pip install watson-machine-learning-client-V4

     |████████████████████████████████| 983kB 312kB/s eta 0:00:01


In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

<a id='create'></a>
### Create a client instance

Use your Watson Machine Learning credentials.

In [4]:
# Instantiate a client using credentials
wml_credentials = {
  "apikey": "dGuBkKsOIA90OIHL_j3IxRkkTUerJJXEeRu5m5r2yhNO",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:eu-de:a/05b089c618b6d54d15a9116170521ef8:9fe12ac2-0282-4468-8466-30ec9b4b14dd::",
  "iam_apikey_name": "auto-generated-apikey-59eb1604-0716-4382-b06c-7c0ac31ec13d",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/05b089c618b6d54d15a9116170521ef8::serviceid:ServiceId-d9b8ba66-b3a2-46b8-b334-1bebb3055d95",
  "instance_id": "9fe12ac2-0282-4468-8466-30ec9b4b14dd",
  "password": "3260c811-0cdb-4604-b1bb-8378c4daaf52",
  "url": "https://eu-de.ml.cloud.ibm.com",
  "username": "59eb1604-0716-4382-b06c-7c0ac31ec13d"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.34'

<a id='prepare'></a>
### Prepare your model archive

Put the model.py file in a subdirectory and create a tar.gz file. The model consists of two parts:
* some functions to create an `inputs` dictionary from files and create files from an `outputs` dictionary,
* the real optimization model which uses the inputs and outputs dictionaries.

Use the `write_file` command to write these models to a `main.py` file. 

Use the `tar` command to create a tar archive.

In [6]:
%mkdir model

In [21]:
%%writefile model/main.py

from docplex.util.environment import get_environment
from os.path import splitext
import pandas
from six import iteritems

def get_all_inputs():
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''
    result = {}
    env = get_environment()
    for iname in [f for f in os.listdir('.') if splitext(f)[1] == '.csv']:
        with env.get_input_stream(iname) as in_stream:
            df = pandas.read_csv(in_stream)
            datasetname, _ = splitext(iname)
            result[datasetname] = df
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")
        

Overwriting model/main.py


In [22]:
%%writefile -a model/main.py

# Load CVS files into inputs dictionnary
inputs = get_all_inputs()

#dd-markdown ### Extract data from inputs dictionnary
#dd-markdown In DO for DSX the data is passed as a dictionnary indexed by table names
#dd-cell
import os, pandas as pd

df_campaign = inputs['campaign']
campaigns = df_campaign['id'].values
df_campaign.set_index('id', inplace=True)

df_customer = inputs['customer'] 
customers = df_customer['id'].values
df_customer.set_index('id', inplace=True)

df_candidate = inputs['candidate']
df_candidate.set_index(["Customer","Campaign"], inplace = True)

df_candidate.head()
#dd-markdown ### Import DO package and create a new model
#dd-cell
from docplex.mp.model import Model
mdl = Model("MarketingSmall")
#dd-markdown ### Create decision variables
#dd-markdown We will have one "selection" binary decision variable for each pair of cusomer and campaign  
#dd-cell
selected = mdl.binary_var_matrix(keys1=customers, keys2=campaigns, name="selected")
df_selected = pd.DataFrame({'selected': selected})
df_selected.index.names=['customer', 'campaign']
#dd-markdown ### Create and add objective
#dd-markdown The objective is to maximize expected revenue
#dd-cell
expected_revenue = mdl.sum( selected[customer,campaign] * df_candidate['expected value'][customer,campaign] for customer in customers for campaign in campaigns) 
mdl.add_kpi(expected_revenue, "Expected revenue");
mdl.maximize(expected_revenue);
#dd-markdown ### Create and add constraints
#dd-markdown * For each campaign the number of selected customers is lower than "max customers"
#dd-cell
for campaign in campaigns:
    mdl.add_constraint( mdl.sum( selected[customer,campaign] for customer in customers) <= df_campaign['max customers'][campaign])
#dd-markdown * For each customer, there is at most one selected campaign
#dd-cell
for customer in customers:
    mdl.add_constraint( mdl.sum( selected[customer,campaign] for campaign in campaigns) <= 1)
#dd-cell
mdl.print_information()
#dd-markdown ### Solve the model
#dd-cell
assert mdl.solve(), "!!! Solve of the model fails"
#dd-cell
mdl.report()
#dd-markdown ### Extract solution value for selected variables
#dd-cell
df_selected = df_selected.selected.apply(lambda v: v.solution_value)
df_selected.head()
#dd-markdown ### Build output data frame
#dd-cell
df_selected = df_selected.reset_index()
df_candidate = df_candidate.reset_index();
df_selected['expected revenue'] = df_selected.selected * df_candidate['expected value']

df_selected.head()
#dd-markdown And add it to outputs dictionnary so that it is exported to scenario
#dd-cell
outputs = {}
outputs['selected'] = df_selected
        
# Generate output files
write_all_outputs(outputs)

Appending to model/main.py


In [23]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

<a id='upload'></a>
### Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:
* the tar archive previously created,
* metadata including the model type and runtime

Get the `model_uid`.

In [24]:
# All available meta data properties 

client.repository.ModelMetaNames.show()

------------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME            TYPE  REQUIRED  SCHEMA
NAME                      str   Y
DESCRIPTION               str   N
INPUT_DATA_SCHEMA         dict  N         {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 'string', 'nullable(optional)': 'string'}]}
TRAINING_DATA_REFERENCES  list  N         [{'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'},

In [25]:
# All available runtimes

client.runtimes.list(pre_defined=True)

--------------------------  --------------------------  ------------------------  --------
GUID                        NAME                        CREATED                   PLATFORM
xgboost_0.82-py3.6          xgboost_0.82-py3.6          2019-08-02T06:02:55.531Z  python
xgboost_0.80-py3.6          xgboost_0.80-py3.6          2019-08-02T06:02:52.157Z  python
scikit-learn_0.20-py3.6     scikit-learn_0.20-py3.6     2019-08-02T06:02:45.108Z  python
scikit-learn_0.19-py3.6     scikit-learn_0.19-py3.6     2019-08-02T06:02:41.790Z  python
tensorflow_1.13-py3.6       tensorflow_1.13-py3.6       2019-08-02T06:02:34.144Z  python
tensorflow_1.11-py3.6       tensorflow_1.11-py3.6       2019-08-02T06:02:31.678Z  python
tensorflow_1.5-py3.6        tensorflow_1.5-py3.6        2019-08-02T06:02:12.855Z  python
ai-function_0.1-py3.6       ai-function_0.1-py3.6       2019-07-05T12:02:38.754Z  python
xgboost_0.82-py3            xgboost_0.82-py3            2019-07-05T12:02:33.447Z  python
spss-modeler_18.2

In [26]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "MarketingSmall",
    client.repository.ModelMetaNames.DESCRIPTION: "MarketingSmall",
    client.repository.ModelMetaNames.TYPE: "do-docplex_12.9",
    client.repository.ModelMetaNames.RUNTIME_UID: "do_12.9"    
}

model_details = client.repository.store_model(model='/home/dsxuser/work/model.tar.gz', meta_props=mnist_metadata)

model_uid = client.repository.get_model_uid(model_details)

print( model_uid )

a8eff467-570f-46bd-814a-59ee2f2fc047


<a id='deploy'></a>
### Create a deployment 

Create a batch deployment for the model, providing information such as:
* the maximum number of compute nodes
* the T-shirt size of the compute nodes

Get the `deployment_uid`.

In [27]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "MarketingSmall Deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "MarketingSmall Deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.COMPUTE: {'name': 'S', 'nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

print( deployment_uid )



#######################################################################################

Synchronous deployment creation for uid: 'a8eff467-570f-46bd-814a-59ee2f2fc047' started

#######################################################################################


ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='70cd7e87-af6f-4a72-b12a-b74b2e03057a'
------------------------------------------------------------------------------------------------


70cd7e87-af6f-4a72-b12a-b74b2e03057a


In [28]:
# List all existing deployments

client.deployments.list()

------------------------------------  -------------------------  -----  ------------------------  -------------
GUID                                  NAME                       STATE  CREATED                   ARTIFACT_TYPE
70cd7e87-af6f-4a72-b12a-b74b2e03057a  MarketingSmall Deployment  ready  2019-08-14T13:47:35.196Z  model
0c2372bf-436d-41f3-b4a4-daeadefff192  MarketingSmall Deployment  ready  2019-08-14T13:43:59.005Z  model
e12f8103-7eb6-4f1c-b650-e779000e4bbd  deploy2                    ready  2019-01-11T10:50:08.771Z  model
ecbae8be-a340-4255-bc66-02e7040019bf  PredictDeployedSvc         ready  2019-01-08T14:03:49.576Z  model
------------------------------------  -------------------------  -----  ------------------------  -------------


<a id='job'></a>
### Create and monitor a job with inline data for your deployed model

Create a payload containing inline input data.

Create a new job with this payload and the deployment.

Get the `job_uid`.

In [29]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_5aea6b8defe6477eb3bb359a3ad797c8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vyscBetMpPlUTXIy_oGwoM89y3Ta7C_-kAOS8SzbMFD7',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_5aea6b8defe6477eb3bb359a3ad797c8.get_object(Bucket='do4wsmarketingcampaignshandson-donotdelete-pr-17nh3lvgn7be59',Key='candidate.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

candidate = pd.read_csv(body)
candidate.head()

body = client_5aea6b8defe6477eb3bb359a3ad797c8.get_object(Bucket='do4wsmarketingcampaignshandson-donotdelete-pr-17nh3lvgn7be59',Key='customer.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

customer = pd.read_csv(body)
customer.head()

body = client_5aea6b8defe6477eb3bb359a3ad797c8.get_object(Bucket='do4wsmarketingcampaignshandson-donotdelete-pr-17nh3lvgn7be59',Key='campaign-1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

campaign = pd.read_csv(body)
campaign.head()

,id,max customers
0,Home,3
1,Auto,3
2,Travel,3


In [30]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id":"customer.csv",
            "values" : customer
        },
        {
            "id":"campaign.csv",
            "values" : campaign
        },
        {
            "id":"candidate.csv",
            "values" : candidate
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.csv"
    }
    ]
}

job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print( job_uid )

9b5eb83c-2e53-4a05-bb7a-6f85d22f9c2b


Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [31]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
queued...
running...
completed


In [36]:
job_details['entity']['decision_optimization']['solve_state']

{'details': {'PROGRESS_GAP': '0.0',
  'MODEL_DETAIL_NONZEROS': '60',
  'MODEL_DETAIL_TYPE': 'MILP',
  'MODEL_DETAIL_CONTINUOUS_VARS': '0',
  'MODEL_DETAIL_CONSTRAINTS': '13',
  'KPI.Expected revenue': '770',
  'PROGRESS_CURRENT_OBJECTIVE': '770',
  'MODEL_DETAIL_INTEGER_VARS': '0',
  'MODEL_DETAIL_KPIS': '["Expected revenue"]',
  'MODEL_DETAIL_BOOLEAN_VARS': '30',
  'PROGRESS_BEST_OBJECTIVE': '770.0'},
 'solve_status': 'optimal_solution'}

<a id='display'></a>
### Extract and display solution

Display the output solution.

Display the KPI Total Calories value.

In [37]:
# Create a dataframe for the solution
solution = pd.DataFrame(job_details['entity']['decision_optimization']['output_data'][0]['values'], 
                        columns = job_details['entity']['decision_optimization']['output_data'][0]['fields'])
solution.head()

,NAME,VALUE
0,Expected revenue,770


### Delete the deployment

Use the following method to delete the deployment.

In [38]:
client.deployments.delete(deployment_uid)

'SUCCESS'

<a id='summary'></a>
### Summary and next steps

You successfully completed this notebook! 

You've learned how to:

- work with the Watson Machine Learning client
- prepare your model archive and upload your model on Watson Machine Learning
- create a deployment
- create and monitor a job with inline data for your deployed model
- display the solution

Check out our online documentation at <a href="https://dataplatform.cloud.ibm.com/docs" target="_blank" rel="noopener noreferrer">https://dataplatform.cloud.ibm.com/docs</a> for more samples, tutorials and documentation. 

<hr>
Copyright © 2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>